# georef v.20200629

This jupyter notebook georeferences videos.

To run using **papermill** enter something like this:

papermill georef.ipynb /media/aubrey/9016-4EF8/20200628/georef.ipynb -p DATAPATH '/media/aubrey/9016-4EF8' -p DATE '20200628'

Input files are a video (\*.mp4) recorded using the camera app and a log file (\*.csv) created using the GPSLogger app.

Output file is a csv file containing frame number, UTC timestamp, latitude and longitude.

# Data 

* directory named like /home/aubrey/Desktop/roadside/jupyter notebooks/20200619
  * one or more video files named like 20200619_104803.mp4
  * one GPS log generated by GPSLogger app named like 20200619.csv    

In [70]:
import cv2
import pandas as pd
import numpy as np
import exiftool
from datetime import datetime, timedelta
import pytz
import logging
import glob

In [71]:
# Default values for 3 parameters. May be changed by papermill.
DATAPATH = '/media/aubrey/9016-4EF8'
DATE = '20200628'
FRAME_INTERVAL = 1800

In [72]:
def get_video_start_stop(filename):
    """
    """  
    with exiftool.ExifTool() as et:
        createDate = et.get_tag('QuickTime:CreateDate', filename)
        logging.info(f'Exif Quicktime:CreateDate: {createDate} UTC')
        duration = et.get_tag('QuickTime:Duration', filename)
        logging.info(f'Exif QuickTime:Duration: {duration} seconds')
    createDate = datetime.strptime(createDate, '%Y:%m:%d %H:%M:%S')   
    start = createDate - timedelta(seconds=duration)
    logging.info(f'Video started at {start} UTC')
    stop = createDate
    logging.info(f'Video ended at {stop} UTC')
    return start, stop

#get_video_start_stop(filename)

In [73]:
def get_lat_lon(timestamp):
    timestamp = pd.to_datetime(timestamp)
    df = dfgps[dfgps.time==timestamp]
    if not df.empty:
        # There is a record for exact timestamp (unlikely); return lat lon.
        return dfgps.lat.values[0], dfgps.lon.values[0]
    else:
        # Estimate lat lon using linear interpolation records just prior and post timestamp 
        df1 = dfgps[dfgps.time<timestamp].tail(1)     
        df2 = dfgps[dfgps.time>timestamp].head(1)
        t1 = df1.time.values[0]
        t2 = df2.time.values[0]
        lat1 = df1.lat.values[0]
        lat2 = df2.lat.values[0]
        lon1 = df1.lon.values[0]
        lon2 = df2.lon.values[0] 
        fraction = (timestamp-t1)/(t2-t1)
        lat = lat1 + fraction*(lat2-lat1)
        lon = lon1 + fraction*(lon2-lon1)
        return lat, lon

#start, stop = get_video_start_stop(filename)    
#get_lat_lon(start)

In [74]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(funcName)s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    handlers=[
        #logging.FileHandler(f'{DATAPATH}/{DATE}/georef.log'),
        logging.StreamHandler(),
    ]
)
logging.info('Starting georef.ipynb v.20200629')

videolist = glob.glob(f'{DATAPATH}/{DATE}/*.mp4')
logging.info(f'videolist = {videolist}')
gpslogpath = glob.glob(f'{DATAPATH}/{DATE}/{DATE}.csv')[0]
logging.info(f'gpslogpath = {gpslogpath}')

logging.info(f'Reading GPS log from {gpslogpath}')
dfgps = pd.read_csv(gpslogpath, parse_dates=['time'])
dfgps['time'] = dfgps['time'].dt.tz_localize(None)
gpslog_start_time = dfgps.time.min().to_pydatetime()
gpslog_stop_time = dfgps.time.max().to_pydatetime()

for video in videolist:
    #frameinterval = FRAME_INTERVAL
    start_time, stop_time = get_video_start_stop(video)    
    if (start_time > gpslog_start_time) and (stop_time < gpslog_stop_time):
        logging.info(f'Started processing {video}')
        logging.info('Building timestamp-location table.')
        data = list()
        cap = cv2.VideoCapture(video)
        i = 0
        while(cap.isOpened()):
            frame_exists, curr_frame = cap.read()
            if frame_exists:
                pos_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp = start_time + timedelta(milliseconds=pos_msec)
                lat, lon = get_lat_lon(timestamp)
                data.append([i, timestamp, lat, lon])
                i += 1
                if (i%FRAME_INTERVAL==0):
                    logging.info(f'{i} {timestamp} {lat:7.4f} {lon:8.4f}')
            else:
                break
        cap.release()
        dfts = pd.DataFrame(data, columns=['frame', 'timestamp', 'lat', 'lon'])
        outputpath = video.replace('.mp4','_gps.csv')
        logging.info(f'Saving output to {outputpath}')                               
        dfts.to_csv(outputpath, index=False)
        logging.info(f'Finished processing {video}')
    else:
        logging.error(f'Cannot process {video} - gps log data not available')
        
logging.info('FINISHED ALL')    

2020-06-29T08:14:34+1000 [INFO] <module> Starting georef.ipynb v.20200629
2020-06-29T08:14:34+1000 [INFO] <module> videolist = ['/media/aubrey/9016-4EF8/20200628/20200628_133929.mp4', '/media/aubrey/9016-4EF8/20200628/20200628_133648.mp4', '/media/aubrey/9016-4EF8/20200628/20200628_133322.mp4', '/media/aubrey/9016-4EF8/20200628/20200628_133119.mp4']
2020-06-29T08:14:34+1000 [INFO] <module> gpslogpath = /media/aubrey/9016-4EF8/20200628/20200628.csv
2020-06-29T08:14:34+1000 [INFO] <module> Reading GPS log from /media/aubrey/9016-4EF8/20200628/20200628.csv
2020-06-29T08:14:34+1000 [INFO] get_video_start_stop Exif Quicktime:CreateDate: 2020:06:28 03:41:58 UTC
2020-06-29T08:14:34+1000 [INFO] get_video_start_stop Exif QuickTime:Duration: 148.369 seconds
2020-06-29T08:14:34+1000 [INFO] get_video_start_stop Video started at 2020-06-28 03:39:29.631000 UTC
2020-06-29T08:14:34+1000 [INFO] get_video_start_stop Video ended at 2020-06-28 03:41:58 UTC
2020-06-29T08:14:34+1000 [INFO] <module> Started 

KeyboardInterrupt: 